In [ ]:
import pandas as pd
import re

#### 탑리뷰어 df

작성자, 주문 날짜, 리뷰 작성 날짜, 리뷰 내용, 리뷰당 ‘도움이 돼요’ 개수, 한달 이상 사용, 재구매 여부, 사진 개수, 제품 구매 방법, 리뷰 고유번호, 평점(긍부정)
리뷰 쓴 주기, 리뷰 길이, 총 ‘도움이 돼요’, 리뷰당 ‘도움이 돼요’ (평균), 체험단 리뷰와 일반 리뷰 차이 (리뷰 길이 편차)

- gdasSeq  :  리뷰 고유번호
- gdasScrVal: 평점 (2,4,6,8,10)
- dispRegDate : 리뷰 업로드 날짜
- gdasCont : 리뷰 본문
- ordDate: 상품 주문 날짜
- photoList: 사진 리스트 (사진 없으면 null)
- recommCnt: 도움이 돼요 개수
- renewUsed1mmGdasYn: 한달이상 사용
- firstGdasYn: 첫 구매 여부 (재구매면 “N” 아니면 “Y”)
- mbrNo: 리뷰를 작성한 회원의 고유 식별자
- gdasSctCd : 제품 구매 방법 (10 - 온라인, 60 - 매장, 50 - 체험단, 70 - 선물)


In [ ]:
# 탑리뷰어 정보 DF로 가져오기
topReviews_path = '/Users/ryeongjoo/Desktop/workspace/semi_prj/topReviews.csv'
top_df = pd.read_csv(topReviews_path, header=0, encoding='UTF-8')

/var/folders/mn/w592whfs2tb4g8s658288fb00000gn/T/ipykernel_9186/3744978623.py:3: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  top_df = pd.read_csv(topReviews_path, header=0, encoding='UTF-8')


In [ ]:
# 필요한 칼럼만 담은 DF
my_top_df = top_df[['mbrNo','ordDate','dispRegDate','gdasCont','recommCnt','renewUsed1mmGdasYn','firstGdasYn', 'gdasSeq', 'gdasScrVal','gdasSctCd' ]]
# print(my_top_df.head())

In [ ]:
# NaN 값 드랍
my_top_df = my_top_df.dropna(subset=['gdasCont'])

# 영어 소문자 통일
my_top_df['gdasCont'] = my_top_df['gdasCont'].str.lower()

In [ ]:
# 정규식을 이용하여 이모티콘 추출하는 함수
def extract_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001f600-\U0001f64f"  # 이모티콘
        u"\U0001f300-\U0001f5ff"  # 기호 및 문장 부호
        u"\U0001f680-\U0001f6ff"  # 기타 이모티콘
        u"\U0001f1e0-\U0001f1ff"  # 국기 및 기호
                           "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

# '이모티콘' 열 생성
my_top_df['unicode'] = my_top_df['gdasCont'].apply(lambda x: extract_emoji(x))

In [ ]:
# 리뷰내용 중복문자, 이모티콘 제거
clean_reviews = []
emoticon_list = []

def remove_emoji(text):
    # 이모티콘 패턴 정의
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 이모티콘
                               u"\U0001F300-\U0001F5FF"  # 심볼
                               u"\U0001F680-\U0001F6FF"  # 트랜스포트 및 지도
                               u"\U0001F1E0-\U0001F1FF"  # 국기 및 이모지
                               "]+", flags=re.UNICODE)
    # 이모티콘 제거
    return emoji_pattern.sub(r'', text)

for review in my_top_df['gdasCont'] :
    
    # 초성, 줄바꿈 제거
    review = re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', review)
    review = re.sub('[\r\n]', ' ', review)  
    
    # 이모티콘 제거
    review = remove_emoji(review)
    
    clean_reviews.append(review)
    
my_top_df['gdasCont'] = clean_reviews

In [ ]:
# 리뷰 길이 칼럼 추가
my_top_df['cont_length'] = my_top_df['gdasCont'].apply(lambda x: len(str(x)))

# print(my_top_df.head())

# 이모티콘 개수 칼럼 추가
my_top_df['emoCnt'] = my_top_df['unicode'].apply(len)

In [ ]:
# 리뷰 주기 칼럼
# 리뷰 작성 날짜 - 구매일자

# 날짜형으로 변환
my_top_df['dispRegDate'] = pd.to_datetime(my_top_df['dispRegDate'])
my_top_df['ordDate'] = pd.to_datetime(my_top_df['ordDate'])

# NaN이 아닌 행 선택
subset = my_top_df[my_top_df['ordDate'].notnull() ]

# 두 날짜의 차이를 계산하여 새로운 칼럼에 추가
my_top_df.loc[subset.index, 'or_diff'] = my_top_df['dispRegDate'] - my_top_df['ordDate']

# 결과 출력
# print(my_top_df.head())

In [ ]:
my_top_df['pn'] = [1 if x > 6 else 0 for x in my_top_df['gdasScrVal']]

#### 리뷰어별 df

In [ ]:
reviewer_df = pd.DataFrame()

In [ ]:
# mbrNo 칼럼 생성
unique_mbrNo = my_top_df['mbrNo'].unique()  # 중복을 제거한 mbrNo 값 추출
reviewer_df['mbrNo'] = pd.Series(unique_mbrNo)

In [ ]:
# 리뷰어별 평균 리뷰 길이
# 총 리뷰 길이 평균 계산
contlen_df = my_top_df.groupby('mbrNo')['cont_length'].mean().reset_index().rename(columns={'cont_length': 'contlen_mean'})

# reviewer_df와 contlen_df 조인
reviewer_df = pd.merge(reviewer_df, contlen_df, on='mbrNo', how='left')

In [ ]:
# 리뷰어별 총 도움이 돼요 개수

# 총 리뷰 개수 계산
recommSum_df = my_top_df.groupby('mbrNo')['recommCnt'].sum().reset_index().rename(columns={'recommCnt': 'total_recomm'})

# reviewer_df와 recommSum_df 조인
reviewer_df = pd.merge(reviewer_df, recommSum_df, on='mbrNo', how='left')

In [ ]:
# 리뷰어별 작성 리뷰 개수
reviewCnt_df = my_top_df.groupby('mbrNo')['gdasCont'].count().reset_index().rename(columns={'gdasCont': 'gdas_cnt'})
reviewer_df = pd.merge(reviewer_df, reviewCnt_df, on='mbrNo', how='left')

# 리뷰어별 리뷰 개수당 평균 도움이 돼요 개수
reviewer_df['recomm_mean'] = reviewer_df['total_recomm'] / reviewer_df['gdas_cnt']

In [ ]:
# 리뷰어별 체험단 리뷰와 일반 리뷰 차이 (리뷰 길이 편차)

# 체험단 리뷰길이 평균
test_df = my_top_df.loc[my_top_df['gdasSctCd'] == 50].groupby('mbrNo')['cont_length'].mean()

# 체험단 제외한 리뷰 길이 평균
buy_df = my_top_df.loc[my_top_df['gdasSctCd'] != 50].groupby('mbrNo')['cont_length'].mean()

# 체험단 리뷰길이 - 체험단 제외한 리뷰 길이 
diff_df = pd.DataFrame({'tb_len_diff': test_df - buy_df}).reset_index()

# reviewer_df와 recommSum_df 조인
reviewer_df = pd.merge(reviewer_df, diff_df, on='mbrNo', how='left')

In [ ]:
# 평균 이모티콘 개수
emoCntMean_df = my_top_df.groupby('mbrNo')['emoCnt'].mean().reset_index().rename(columns={'emoCnt': 'emoCntMean'})
reviewer_df = pd.merge(reviewer_df, emoCntMean_df, on='mbrNo', how='left')

In [ ]:
# 평점 긍부정 경향
scrPN_df = my_top_df.groupby('mbrNo')['pn'].mean().reset_index().rename(columns={'pn': 'scrPN'})
reviewer_df = pd.merge(reviewer_df, scrPN_df, on='mbrNo', how='left')

In [ ]:
reviewer_df.head()

,mbrNo,contlen_mean,total_recomm,gdas_cnt,recomm_mean,tb_len_diff,emoCntMean,topRvrRnk,topRvrRnk_group,scrPN
0,M0000004535694,83.013333,1018.0,300,3.393333,218.630508,0.343333,1,1,1.000000
1,M0000012163234,101.200000,1216.0,80,15.200000,304.000000,0.250000,2,1,0.850000
2,M0000016082816,110.190909,1177.0,110,10.700000,222.396226,0.009091,3,1,0.945455
3,M0000003453847,73.930000,604.0,100,6.040000,248.250000,0.450000,4,1,0.990000
4,M0000010425691,505.030000,765.0,100,7.650000,37.208333,1.560000,5,1,0.960000


#### 상,중,하위 리뷰어 확인

In [ ]:
# 탑리뷰어(1~1000) 상위,중위,하위로 나눠서 리뷰길이차이 (편차) 구하기

reviewer_df['topRvrRnk'] = reviewer_df['mbrNo'].index+1

#탑리뷰어(1~2000) 상위,중위,하위로 나눈 칼럼 추가
reviewer_df['topRvrRnk_group'] = pd.cut(reviewer_df['topRvrRnk'], bins=[0, 657, 1315, 1972], labels=[1, 2, 3])



-27.177575757575752
83.01333333333334 101.2 110.19090909090909


In [ ]:
# 상위 리뷰어와 하위 리뷰어 평균 작성 리뷰 개수 차이 
high_review_cnt = reviewer_df.loc[reviewer_df['topRvrRnk'] == 1]['gdas_cnt'].mean()
middle_review_cnt = reviewer_df.loc[reviewer_df['topRvrRnk'] == 2]['gdas_cnt'].mean()
low_review_cnt = reviewer_df.loc[reviewer_df['topRvrRnk'] == 3]['gdas_cnt'].mean()

diff_review_cnt = high_review_cnt - low_review_cnt
print(diff_review_cnt)
print(high_review_cnt, middle_review_cnt, low_review_cnt)

# 상위 리뷰어들의 평균 리뷰 개수가 훨씬 많음을 알 수 있다.

190.0
300.0 80.0 110.0


In [ ]:
#상위 리뷰어와 하위 리뷰어 차이 (리뷰 길이 편차)
high_review_length = reviewer_df.loc[reviewer_df['topRvrRnk'] == 1]['contlen_mean'].mean()
middle_review_length = reviewer_df.loc[reviewer_df['topRvrRnk'] == 2]['contlen_mean'].mean()
low_review_length = reviewer_df.loc[reviewer_df['topRvrRnk'] == 3]['contlen_mean'].mean()

diff_review_length = high_review_length - low_review_length
print(diff_review_length)
print(high_review_length, middle_review_length, low_review_length)

# 오히려 하위 탑리뷰어들의 평균 리뷰 길이가 더 길다는 것을 알 수 있다.

In [ ]:
#상위 리뷰어와 하위 리뷰어 이모티콘 개수 차이
high_review_emo = reviewer_df.loc[reviewer_df['topRvrRnk'] == 1]['emoCntMean'].mean()
low_review_emo = reviewer_df.loc[reviewer_df['topRvrRnk'] == 3]['emoCntMean'].mean()

diff_review_emo = high_review_emo - low_review_emo
print(diff_review_emo)

# 상위 탑리뷰어들의 평균 이모티콘 개수가 더 많다는 것을 알 수 있다.

0.33424242424242423


In [ ]:
# 상위 리뷰어와 하위 리뷰어 평점 긍부정 차이 
high_review_pn = reviewer_df.loc[reviewer_df['topRvrRnk'] == 1]['scrPN'].mean()
low_review_pn = reviewer_df.loc[reviewer_df['topRvrRnk'] == 3]['scrPN'].mean()

diff_review_pn = high_review_pn - low_review_pn
print(diff_review_pn)

# 상위 탑리뷰어들이 평균적으로 더 높은 평점을 주었다는 것을 알 수 있다.

0.054545454545454564


In [ ]:
# 상위 리뷰어와 하위 리뷰어 평균 도움이돼요 차이 
high_review_rcm = reviewer_df.loc[reviewer_df['topRvrRnk'] == 1]['recomm_mean'].mean()
middle_review_rcm = reviewer_df.loc[reviewer_df['topRvrRnk'] == 2]['recomm_mean'].mean()
low_review_rcm = reviewer_df.loc[reviewer_df['topRvrRnk'] == 3]['recomm_mean'].mean()

diff_review_rcm = high_review_rcm - low_review_rcm
print(diff_review_rcm)
print(high_review_rcm, middle_review_rcm, low_review_rcm)

# 하위 리뷰어들의 평균 도움이돼요 개수가 더 많다는 것을 알 수 있다

-7.306666666666666
3.3933333333333335 15.2 10.7


In [ ]:
# def text_len(text):
#     return re.sub('[^ㅏ-ㅣ가-힣 .,+…\';:\-ㆍ\(\)&0-9A-Za-z]+', '', text)
# my_top_df['gdasCont'][:10].apply(text_len)

#### 리뷰 내용 텍스트 전처리

In [ ]:
# 토큰화

from konlpy.tag import Okt
import re

tokens_results = []

for text in my_top_df['gdasCont'].head(5):
    
    # Okt 객체 생성
    okt = Okt()

    # 초성, 줄바꿈 제거
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ\r\n]', '', text)

    tokens = okt.pos(text)
    result = []

    for word, tag in tokens:
        if tag not in ['Josa', 'Eomi', 'Punctuation']:
            result.append(word)

    tokens_results.append(result)
    
print(tokens_results)

[['제', '최근', '올리브영', '가장', '많이', '구매', '했던', '제품', '중', '하나', '입니다', '신랑', '30', 'g', '짜', '리', '하루', '두', '봉지', '씩', '먹네요', '하지만', '제입', '많이', '달아요'], ['허브', '쏠트', '믹스넛', '견과류', '구성은', '맘', '드는데', '넘', '짜요', '그래도', '1', '1', '행사', '하길래', '구매', '했는데', '겉', '묻은', '가루', '털어내고', '먹어야', '해'], ['아침', '주로', '마시고', '있는', '사과', '당근', '맛', '이', '너', '주스', '입니다', '제', '5년', '을', '넘게', '마시고', '있는데', '꾸준하게', '마시면', '화장실', '가는게', '편해져서', '좋아요'], ['신랑', '줄려고', '신청', '했어요', '다비도프', '향수', '프랑스', '만들어', '졌고', '향수병', '네모', '형태', '되어있고', '잡기', '편하네요', '제품', '받고', '바로', '신랑', '뿌려', '봤는데', '향', '딱', '남자', '들', '좋아할만', '한', '향', '요', '일단', '향', '중요하지만', '지속', '력', '좋아야', '해서', '신랑', '뿌리', '한두', '시간', '지나서도', '향', '나는지', '옆', '가봤더니', '진하게는', '아니지만', '은은하게', '잔향', '남아', '있는듯', '하네요', '날씨', '점점', '더워지는데', '여름', '뿌려도', '좋을것', '같다는', '생각', '드네', '나이', '상관없이', '모든', '연령', '층', '사용', '하기에', '괜찮은', '제품', '듯', '합니다'], ['팽이버섯', '유산균', '발효', '액', '들어가서', '꾸준하게', '마시면', '화장실', '가는게', '편해지네요']]


In [ ]:
# 키워드 추출
from sklearn.feature_extraction.text import TfidfVectorizer

for documents in tokens_results:

    # TfidfVectorizer 객체 생성
    tfidf_vectorizer = TfidfVectorizer()

    # TfidfVectorizer로 문서 벡터화 수행
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    # get_feature_names() 메서드를 이용해 단어 목록과 각 단어의 인덱스 확인
    feature_names = tfidf_vectorizer.get_feature_names_out()

    print(feature_names)

['30' '가장' '구매' '달아요' '많이' '먹네요' '봉지' '신랑' '올리브영' '입니다' '제입' '제품' '최근'
 '하나' '하루' '하지만' '했던']
['가루' '견과류' '구매' '구성은' '그래도' '드는데' '먹어야' '묻은' '믹스넛' '쏠트' '짜요' '털어내고' '하길래'
 '했는데' '행사' '허브']
['5년' '가는게' '꾸준하게' '넘게' '당근' '마시고' '마시면' '사과' '아침' '입니다' '있는' '있는데' '좋아요'
 '주로' '주스' '편해져서' '화장실']
['가봤더니' '같다는' '괜찮은' '나는지' '나이' '날씨' '남아' '남자' '네모' '다비도프' '더워지는데' '되어있고'
 '드네' '만들어' '모든' '바로' '받고' '봤는데' '뿌려' '뿌려도' '뿌리' '사용' '상관없이' '생각' '시간'
 '신랑' '신청' '아니지만' '여름' '연령' '은은하게' '일단' '있는듯' '잔향' '잡기' '점점' '제품' '졌고'
 '좋아야' '좋아할만' '좋을것' '줄려고' '중요하지만' '지나서도' '지속' '진하게는' '편하네요' '프랑스' '하기에'
 '하네요' '한두' '합니다' '해서' '했어요' '향수' '향수병' '형태']
['가는게' '꾸준하게' '들어가서' '마시면' '발효' '유산균' '팽이버섯' '편해지네요' '화장실']


In [ ]:
# 요약

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import kss
import networkx as nx


for document in my_top_df['gdasCont'].head(5):
    # Okt 객체 생성
    okt = Okt()

    # 문장 단위로 분리
    sentences = kss.split_sentences(document)

    # 각 문장에서 명사, 형용사, 동사만 추출하여 리스트에 저장
    words_list = []
    for sentence in sentences:
        words = okt.pos(sentence, stem=True)
        words = [word[0] for word in words if word[1] in ['Noun', 'Adjective', 'Verb']]
        words_list.append(' '.join(words))

    # TF-IDF를 계산하여 문장 간 유사도를 계산
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(words_list)
    similarity_matrix = cosine_similarity(tfidf, tfidf)

    # TextRank 알고리즘을 적용하여 문서 요약
    def summarize(document, num_sentences=3):


        # 각 문장에서 명사, 형용사, 동사만 추출하여 리스트에 저장
        words_list = []
        for sentence in sentences:
            words = okt.pos(sentence, stem=True)
            words = [word[0] for word in words if word[1] in ['Noun', 'Adjective', 'Verb']]
            words_list.append(' '.join(words))

        # TF-IDF를 계산하여 문장 간 유사도를 계산
        vectorizer = TfidfVectorizer()
        tfidf = vectorizer.fit_transform(words_list)
        similarity_matrix = cosine_similarity(tfidf, tfidf)

        # TextRank 알고리즘을 적용하여 문서 요약
        similarity_graph = similarity_matrix
        nx_graph = nx.from_numpy_array(similarity_graph)
        scores = nx.pagerank(nx_graph)

        # 핵심 문장 추출
        ranked_sentences = sorted(((score, index) for index, score in scores.items()), reverse=True)
        selected_sentences = [sentences[index] for _, index in ranked_sentences[:num_sentences]]
        return ' '.join(selected_sentences)

    # 문서 요약
    summary = summarize(document, num_sentences=3)
    print(summary)


[Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D



하지만 제입엔 많이 달아요 신랑이 30g짜리를 하루에 두봉지씩  먹네요 제가 최근에 올리브영에서 가장 많이  구매를 했던 제품중 하나입니다
그래도 1+1 행사를 하길래 구매를  했는데 겉에 묻은 가루를 털어내고  먹어야 해요 허브쏠트 믹스넛은 견과류구성은  맘에 드는데 넘짜요
제가 5년을 넘게 마시고 있는데 꾸준하게  마시면 화장실 가는게 편해져서 좋아요 아침에 주로 마시고 있는 사과당근맛  이너주스입니다
신랑을 줄려고 신청을 했어요~ 일단 향도 중요하지만 지속력도 좋아야  해서 신랑이 뿌리고 한두시간 지나서도  향이 나는지 옆에 가봤더니 진하게는  아니지만 은은하게 잔향이 남아 있는듯  하네요 나이와 상관없이 모든 연령층이 사용하기에  괜찮은 제품인듯 합니다
팽이버섯 유산균 발효액이 들어가서  꾸준하게 마시면 화장실 가는게  편해지네요


In [ ]:
# 요약2

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import kss
import networkx as nx


for document in my_top_df['gdasCont'].head(5):
    # Okt 객체 생성
    okt = Okt()
    
    # 초성, 줄바꿈 제거
    document = re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', document)
    document = re.sub('[\r\n]', ' ', document)
    
    # 문장 단위로 분리
    sentences = kss.split_sentences(document)
    
    # 각 문장에서 명사, 형용사, 동사만 추출하여 리스트에 저장
    words_list = []
    for sentence in sentences:
        words = okt.pos(sentence, stem=True)
        words = [word[0] for word in words if word[1] in ['Noun', 'Adjective', 'Verb']]
        words_list.append(' '.join(words))

    # TF-IDF를 계산하여 문장 간 유사도를 계산
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(words_list)
    similarity_matrix = cosine_similarity(tfidf, tfidf)

    # TextRank 알고리즘을 적용하여 문서 요약
    def summarize(document, num_sentences=3):


        # 각 문장에서 명사, 형용사, 동사만 추출하여 리스트에 저장
        words_list = []
        for sentence in sentences:
            words = okt.pos(sentence, stem=True)
            words = [word[0] for word in words if word[1] in ['Noun', 'Adjective', 'Verb']]
            words_list.append(' '.join(words))

        # TF-IDF를 계산하여 문장 간 유사도를 계산
        vectorizer = TfidfVectorizer()
        tfidf = vectorizer.fit_transform(words_list)
        similarity_matrix = cosine_similarity(tfidf, tfidf)

        # TextRank 알고리즘을 적용하여 문서 요약
        similarity_graph = similarity_matrix
        nx_graph = nx.from_numpy_array(similarity_graph)
        scores = nx.pagerank(nx_graph)

        # 핵심 문장 추출
        ranked_sentences = sorted(((score, index) for index, score in scores.items()), reverse=True)
        selected_sentences = [sentences[index] for _, index in ranked_sentences[:num_sentences]]
        return ' '.join(selected_sentences)

    # 문서 요약
    summary = summarize(document, num_sentences=3)
    print(summary)


하지만 제입엔 많이 달아요 신랑이 30g짜리를 하루에 두봉지씩  먹네요 제가 최근에 올리브영에서 가장 많이  구매를 했던 제품중 하나입니다
그래도 1+1 행사를 하길래 구매를  했는데 겉에 묻은 가루를 털어내고  먹어야 해요 허브쏠트 믹스넛은 견과류구성은  맘에 드는데 넘짜요
제가 5년을 넘게 마시고 있는데 꾸준하게  마시면 화장실 가는게 편해져서 좋아요 아침에 주로 마시고 있는 사과당근맛  이너주스입니다
신랑을 줄려고 신청을 했어요~ 일단 향도 중요하지만 지속력도 좋아야  해서 신랑이 뿌리고 한두시간 지나서도  향이 나는지 옆에 가봤더니 진하게는  아니지만 은은하게 잔향이 남아 있는듯  하네요 나이와 상관없이 모든 연령층이 사용하기에  괜찮은 제품인듯 합니다
팽이버섯 유산균 발효액이 들어가서  꾸준하게 마시면 화장실 가는게  편해지네요


In [ ]:
# 이모티콘
import unicodedata
print(unicodedata.name('😂'))

FACE WITH TEARS OF JOY
